In [1]:
%cd /content/drive/MyDrive/yolo_workspace/yolov34/darknet

/content/drive/MyDrive/yolo_workspace/yolov34/darknet


In [2]:
import os
import cv2
import sys
import glob
import time

VERSION = 4
config_file = f"BCCD/yolov{VERSION}_test.cfg"
data_file = "BCCD/obj.data"
weights = f"BCCD/weights/yolov{VERSION}_train_best.weights"
thresh = 0.5

sys.path.append("/content/drive/MyDrive/yolo_workspace/yolov34/darknet/build/darknet/x64")
import darknet

In [7]:
class Yolo:

    def __init__(self):
        # load network
        # os.chdir(darknet_dir)
        self.network, self.class_names, _ = darknet.load_network(
            config_file, data_file, weights, batch_size=1)
        self.w_net = darknet.network_width(self.network)
        self.h_net = darknet.network_height(self.network)

    def detect_object_multi(self, img):
        # make image for object detection
        t0 = time.time()
        img_for_detection = darknet.make_image(self.w_net, self.h_net, 3)
        frame_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frame_resized = cv2.resize(frame_rgb, (self.w_net, self.h_net),
                                   interpolation=cv2.INTER_LINEAR)
        darknet.copy_image_from_bytes(img_for_detection, frame_resized.tobytes())
        detections = darknet.detect_image(self.network, self.class_names, img_for_detection,
                                          thresh=thresh)
        inf_time = time.time() - t0

        # # data processing
        res = []
        for label, conf, bbox in detections:
            cx, cy, w, h = bbox
            h_img, w_img = img.shape[:2]
            xmin = int((cx - (w / 2)) * (w_img / self.w_net))
            xmax = int((cx + (w / 2)) * (w_img / self.w_net))
            ymin = int((cy - (h / 2)) * (h_img / self.h_net))
            ymax = int((cy + (h / 2)) * (h_img / self.h_net))
            res.append([label, conf, (xmin, ymin, xmax, ymax)])
        return res, inf_time

In [8]:
yolo = Yolo()
dir_ref = "BCCD/test"
inf_times = []
for img_path in glob.glob(f"{dir_ref}/*.jpg"):
    img = cv2.imread(img_path)
    res, inf_time = yolo.detect_object_multi(img)
    inf_times.append(inf_time)
    for label, conf, bbox in res:
        xmin, ymin, xmax, ymax = bbox
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(img, label, (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2)
        cv2.imwrite(img_path.replace("test", f"results{VERSION}"), img)

In [9]:
inf_times

[0.036179304122924805,
 0.03566431999206543,
 0.03566861152648926,
 0.03587961196899414,
 0.035666465759277344,
 0.03582334518432617,
 0.03598475456237793,
 0.03577303886413574,
 0.03558683395385742,
 0.03560042381286621,
 0.035875558853149414,
 0.03573727607727051,
 0.03737235069274902,
 0.0357966423034668,
 0.0359344482421875,
 0.03543353080749512,
 0.035635948181152344,
 0.03577399253845215,
 0.035567283630371094,
 0.03567051887512207,
 0.03579258918762207,
 0.03557610511779785,
 0.03566241264343262,
 0.03560328483581543,
 0.035559892654418945,
 0.03633737564086914,
 0.03611326217651367,
 0.035666704177856445,
 0.03592681884765625,
 0.03545880317687988,
 0.03540396690368652,
 0.03567361831665039,
 0.03548455238342285,
 0.03562593460083008,
 0.035810232162475586,
 0.03553962707519531]

In [10]:
print(sum(inf_times) / len(inf_times))

0.0357738733291626


yolov3: 0.029413832558525935

yolov4: 0.0357738733291626